## Create `Model` Class
Must have `setup()` and `predict()` methods.

In [116]:
class PioModel(object):

    def __init__(self, 
                 request_transformer, 
                 response_transformer):

        self.request_transformer = request_transformer
        self.response_transformer = response_transformer
        
    def setup(self):
        pass
    
    def predict(self, 
                request):
        pass

In [117]:
class Model(PioModel):

    def __init__(self, 
                 request_transformer, 
                 response_transformer,                 
                 cat_mean, 
                 cat_stdv, 
                 dog_mean, 
                 dog_stdv):
        PioModel.__init__(self,
                          request_transformer,
                          response_transformer)
        self.cat_mean = cat_mean
        self.cat_stdv = cat_stdv
        self.dog_mean = dog_mean
        self.dog_stdv = dog_stdv

    def transform_request(self, 
                          request):
        return self.request_transformer.transform_request(request)

    def transform_response(self, 
                           response):
        return self.response_transformer.transform_response(response)
        
    def setup(self):
        pass

    def predict(self, 
                request):
        cat_affinity_score = sum([ d['weight'] * d['user_score'] for d in request if 'cat' in d['tags'] ])
        dog_affinity_score = sum([ d['weight'] * d['user_score'] for d in request if 'dog' in d['tags'] ])

        # create normalized z score for compare/classify
        cat_zscore = (cat_affinity_score - self.cat_mean)/self.cat_stdv
        dog_zscore = (dog_affinity_score - self.dog_mean)/self.dog_stdv

        # classify
        if abs(cat_zscore) > abs(dog_zscore):
            if cat_zscore >= 0:
                category = "cat_lover"
            else:
                category = "cat_hater"
        else:
            if dog_zscore >= 0:
                category = "dog_lover"
            else:
                category = "dog_hater"

        classification = {
            'category': category,
            'cat_affinity_score': cat_affinity_score,
            'dog_affinity_score': dog_affinity_score,
            'cat_zscore': cat_zscore,
            'cat_zscore': dog_zscore
        }

        return classification

## Create Request Transformer
Must have a `transform_request()` method.

In [118]:
class PioRequestTransformer(object):

    def transform_request(self,
                          request):
        pass

### Convert `json` -> `dict`

In [119]:
class RequestTransformer(PioRequestTransformer):    

    def transform_request(self,
                          request):

        request_str = request.decode('utf-8')
        request_str = request_str.strip().replace('\n', ',')
        request_dict = json.loads(request_str)
        
        return request_dict
    
    

## Create Response Transformer
Must have a `transform_response()` method.

In [120]:
class PioResponseTransformer(object):

    def transform_response(self,
                           response):
        pass

### Convert `dict` -> `json`

In [121]:
class ResponseTransformer(PioResponseTransformer):

    def transform_response(self,
                           response):
        response_json = json.dumps(response)
        return response_json

## Train Model

In [122]:
cat_mean = 0.1
cat_stdv = 0.20
dog_mean = 0.3
dog_stdv = 0.40

model = Model(RequestTransformer(), 
              ResponseTransformer(), 
              cat_mean, 
              cat_stdv, 
              dog_mean, 
              dog_stdv)

## Pickle Model `model.pkl`

In [123]:
import dill as pickle

model_pkl_path = 'model.pkl'

with open(model_pkl_path, 'wb') as fh:
    pickle.dump(model, fh)